In [1]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.6f}".format

train_df = pd.read_csv(filepath_or_buffer="Z_expectation/Z_values_100000.csv")
test_df = pd.read_csv(filepath_or_buffer="Z_expectation/Z_values_100.csv")
validation_df = pd.read_csv(filepath_or_buffer="Z_expectation/Z_values_10000.csv")

train_df.head()

,0_0,0_1,1_0,1_1,2_0,2_1,3_0,3_1,4_0,4_1,...,7_1,8_0,8_1,9_0,9_1,decay_0,decay_1,W_0,W_1,J_0
0,1.000000,1.000000,0.820000,0.320000,0.040000,-0.340000,-0.160000,-0.200000,-0.020000,-0.080000,...,-0.720000,-0.400000,0.260000,-0.760000,0.800000,1.288213,-0.028388,1.334944,3.454920,4.611197
1,1.000000,1.000000,0.700000,0.600000,-0.040000,0.140000,-0.420000,-0.280000,-0.260000,-0.260000,...,-0.120000,-0.580000,-0.480000,-0.560000,-0.720000,0.979878,2.065299,2.823135,2.489759,3.153725
2,1.000000,1.000000,0.520000,0.400000,-0.020000,-0.400000,-0.380000,-0.380000,-0.140000,-0.100000,...,-0.320000,-0.760000,0.460000,0.200000,1.000000,0.438522,-0.289307,3.004403,3.934976,4.238538
3,1.000000,1.000000,0.640000,0.760000,-0.020000,0.380000,-0.580000,-0.460000,-0.700000,-0.720000,...,0.000000,-0.400000,-0.260000,-0.280000,-0.700000,1.591963,1.266298,3.310940,2.432493,2.903198
4,1.000000,1.000000,0.580000,0.620000,-0.360000,-0.280000,-0.440000,-0.460000,-0.300000,-0.420000,...,-0.700000,-0.200000,-0.320000,0.540000,0.080000,1.819276,0.291896,3.732673,3.377953,3.459519


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


In [3]:
timesteps = 10  # Corrected number of timesteps
# Correcting the extraction of the sequences to include only the first 10 timesteps
sequence_0_corrected = train_df[[f"{i}_0" for i in range(timesteps)]]
sequence_1_corrected = train_df[[f"{i}_1" for i in range(timesteps)]]

# Reshaping the sequences for LSTM input
# The new shape will be [samples, timesteps, features]
samples = len(train_df)  # Number of samples

sequence_0_reshaped = sequence_0_corrected.values.reshape(samples, timesteps, 1)
sequence_1_reshaped = sequence_1_corrected.values.reshape(samples, timesteps, 1)

# Stack the sequences together along the last dimension
train_features = np.concatenate([sequence_0_reshaped, sequence_1_reshaped], axis=-1)

#
sequence_0_corrected_test = test_df[[f"{i}_0" for i in range(timesteps)]]
sequence_1_corrected_test = test_df[[f"{i}_1" for i in range(timesteps)]]

samples = len(test_df)  # Number of samples
sequence_0_reshaped_test = sequence_0_corrected_test.values.reshape(samples, timesteps, 1)
sequence_1_reshaped_test = sequence_1_corrected_test.values.reshape(samples, timesteps, 1)

test_features = np.concatenate([sequence_0_reshaped_test, sequence_1_reshaped_test], axis=-1)

In [ ]:
from keras.src.preprocessing.sequence import TimeseriesGenerator
from tensorflow.python.keras.regularizers import L2, L1
import re
learning_rate = 0.001
epochs = 15
batch_size = 1

# Get all column names as a list
keys = train_df.keys().tolist()

# Remove specific columns from the list
#keys = [key for key in keys if key not in ['W_0', 'W_1', 'J_0']]

#inputs = {key:tf.keras.layers.Input(shape=(1,), name=key) for key in keys}
#concatenated_inputs = tf.keras.layers.concatenate(inputs.values())
# Prepare data for training
#train_features = {key: train_df[key] for key in inputs}
train_labels = train_df[['W_0', 'W_1', 'J_0']]

# Similarly prepare test and validation data
#test_features = {key: test_df[key] for key in inputs}
test_labels = test_df[['W_0', 'W_1', 'J_0']]
#
#validation_features = {key: validation_df[key] for key in inputs}
validation_labels = validation_df[['W_0', 'W_1', 'J_0']]


n_input = 3  # Number of lag observations as input (this can be tuned)
n_features = 10  # Number of features

train_generator = TimeseriesGenerator(train_features, train_labels, length=n_input, batch_size=batch_size)
test_generator = TimeseriesGenerator(test_features, test_labels, length=n_input, batch_size=batch_size)
# Define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(3))  # Output layer for 3 parameters w_0, w_1, J_0
model.compile(optimizer='adam', loss='mse')

# Train the model



In [5]:
history = model.fit(train_generator, epochs=epochs)  # Number of epochs can be tuned
# Create a test generator

# Evaluate the model
model.evaluate(test_generator)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

KeyError: 3

In [33]:
# Training parameters
epochs = 100
batch_size = 1000

# Train the model
history = model.fit(z_sequences, target_sequences, epochs=epochs, batch_size=batch_size, validation_split=0.2)
# Extract loss and validation loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']

Epoch 1/100


ValueError: in user code:

    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\

In [ ]:
epochs_range = range(epochs)

# Plotting
plt.figure(figsize=(8, 8))
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
#plt.xlim([1,epochs])
#plt.ylim([0,2])
plt.semilogy()
plt.show()